In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import os
import sys
from itertools import product

In [ ]:
sys.path.insert(0, os.getcwd() + "/../scripts")
import _helpers as h

plt.style.use(["bmh", "../matplotlibrc"])
%matplotlib inline

In [ ]:
with open("../config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [ ]:
with open("../config.pypsaeur.yaml", 'r') as stream:
    peur_config = yaml.safe_load(stream)
    TECH_COLORS = peur_config["plotting"]["tech_colors"]
    NICE_NAMES = peur_config["plotting"]["nice_names"]

In [ ]:
dimensions = ["solar", "wind", "transmission", "H2", "battery"]
senses = ["min", "max"]
for dimension, sense in product(dimensions, senses):
    epsilons = [0.01, 0.02,0.04,0.06,0.08]

    error_data = {}
    error_data[0.0] = pd.read_csv(f"../results/pce/test-errors-min-cost-0.0.csv", index_col=0)
    for epsilon in epsilons:
        error_data[epsilon] = pd.read_csv(f"../results/pce/test-errors-{sense}-{dimension}-{epsilon}.csv", index_col=0)

    error_data = pd.concat(error_data, axis=0, names=["epsilon", "carrier"])

    for error in ["r2", "mape", "mae"]:

        fig, ax = plt.subplots(figsize=(4,3))

        df = error_data[error].unstack("carrier")

        c = df.columns.map(TECH_COLORS)
        df.columns = df.columns.map(NICE_NAMES)
        df.index = [100*i for i in df.index]

        df.plot(ax=ax, color=c)
        plt.box(False)
        plt.grid(False)
        plt.legend(frameon=False, bbox_to_anchor=(1.01,1.01))
        plt.xlabel(r"$\varepsilon$ [%]")
        plt.ylabel(error.upper())

        ylim = dict(r2=[0.75,1.02], mape=[-1,20], mae=[-1,35], variance_explained=[0.75,1.02])
        plt.ylim(ylim[error])
        
        plt.title(f"{sense.capitalize()} {NICE_NAMES[dimension]}")

        plt.savefig(f"graphics/errorepsilon/ee-{sense}-{dimension}-{error}.pdf", bbox_inches='tight')
        
        plt.close(fig)